# Fast Fourier Transform (FFT) derivation

In [2]:
%pip install sympy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 385.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 425.9 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


## FFT dervation

## FFT Decomposition Proof

We'll demonstrate how the Discrete Fourier Transform (DFT) can be decomposed recursively using the divide-and-conquer approach, which leads to the Fast Fourier Transform (FFT) algorithm.

Starting with the DFT definition:

$$X[k] = \sum_{n=0}^{N-1} x[n]e^{-i\frac{2\pi kn}{N}}$$

We can split this sum into even and odd indices:

$$X[k] = \sum_{m=0}^{\frac{N}{2}-1} x[2m]e^{-i\frac{2\pi k(2m)}{N}} + \sum_{m=0}^{\frac{N}{2}-1} x[2m+1]e^{-i\frac{2\pi k(2m+1)}{N}}$$

Let's simplify the exponential terms:

$$e^{-i\frac{2\pi k(2m)}{N}} = e^{-i\frac{2\pi km}{N/2}} \quad \text{and} \quad e^{-i\frac{2\pi k(2m+1)}{N}} = e^{-i\frac{2\pi km}{N/2}} \cdot e^{-i\frac{2\pi k}{N}}$$

This gives us:

$$X[k] = \sum_{m=0}^{\frac{N}{2}-1} x[2m]e^{-i\frac{2\pi km}{N/2}} + e^{-i\frac{2\pi k}{N}}\sum_{m=0}^{\frac{N}{2}-1} x[2m+1]e^{-i\frac{2\pi km}{N/2}}$$

Now we can define:
- $X_{\text{even}}[k] = \sum_{m=0}^{\frac{N}{2}-1} x[2m]e^{-i\frac{2\pi km}{N/2}}$ (DFT of even-indexed elements)
- $X_{\text{odd}}[k] = \sum_{m=0}^{\frac{N}{2}-1} x[2m+1]e^{-i\frac{2\pi km}{N/2}}$ (DFT of odd-indexed elements)

This gives us the key FFT decomposition formula:

$$X[k] = X_{\text{even}}[k] + e^{-i\frac{2\pi k}{N}} \cdot X_{\text{odd}}[k]$$

For $k + \frac{N}{2}$, using the periodicity property:

$$X[k+\frac{N}{2}] = X_{\text{even}}[k] + e^{-i\frac{2\pi (k+N/2)}{N}} \cdot X_{\text{odd}}[k]$$

Since $e^{-i\frac{2\pi (k+N/2)}{N}} = e^{-i\frac{2\pi k}{N}} \cdot e^{-i\pi} = -e^{-i\frac{2\pi k}{N}}$, we get:

$$X[k+\frac{N}{2}] = X_{\text{even}}[k] - e^{-i\frac{2\pi k}{N}} \cdot X_{\text{odd}}[k]$$

This allows us to compute the full DFT using only half the operations, and by applying this recursively, we achieve the $O(N \log N)$ complexity of the FFT algorithm.

In [9]:
import sympy as sp
from IPython.display import display, Math

# Define symbols
n, m, k, X_even_k, X_odd_k = sp.symbols('n m k X_even_k X_odd_k', integer=True)
N = sp.Symbol('N', integer=True, positive=True)
x = sp.IndexedBase('x')
X = sp.IndexedBase('X')

# Original DFT formula
X_k_formula = sp.Sum(x[n] * sp.exp(-2j * sp.pi * k * n / N), (n, 0, N-1))
X_k_eq = sp.Eq(X[k], X_k_formula)

print("Original DFT formula:")
display(X_k_eq)

# Split into even and odd indices
N_half = N/2
X_k_even = sp.Sum(x[2*m] * sp.exp(-2j * sp.pi * k * 2 * m / N), (m, 0, N_half-1))
X_k_odd = sp.Sum(x[2*m+1] * sp.exp(-2j * sp.pi * k * (2*m+1) / N), (m, 0, N_half-1))
X_k_split = X_k_even + X_k_odd

print("\nSplit into even and odd indices:")
display(sp.Eq(X[k], X_k_split))

# Simplify the exponential terms
X_k_even_simplified = sp.Sum(x[2*m] * sp.exp(-2j * sp.pi * k * m / (N/2)), (m, 0, N_half-1))
X_k_odd_simplified = sp.exp(-2j * sp.pi * k / N) * sp.Sum(x[2*m+1] * sp.exp(-2j * sp.pi * k * m / (N/2)), (m, 0, N_half-1))

print("\nSimplified exponential terms:")
display(sp.Eq(X[k], X_k_even_simplified + X_k_odd_simplified))

# Define the smaller DFTs
print("\nDefining smaller DFTs:")
X_even_k_def = sp.Eq(X_even_k, sp.Sum(x[2*m] * sp.exp(-2j * sp.pi * k * m / (N/2)), (m, 0, N_half-1)))
X_odd_k_def = sp.Eq(X_odd_k, sp.Sum(x[2*m+1] * sp.exp(-2j * sp.pi * k * m / (N/2)), (m, 0, N_half-1)))
display(X_even_k_def)
display(X_odd_k_def)

# Final FFT recurrence relation
twiddle_factor = sp.exp(-2j * sp.pi * k / N)
X_k_recurrence = sp.Eq(X[k], X_even_k + twiddle_factor * X_odd_k)
print("\nFFT recurrence relation:")
display(X_k_recurrence)

# For k + N/2
k_plus_half = k + N/2
twiddle_factor_half = sp.exp(-2j * sp.pi * k_plus_half / N)
twiddle_factor_half_simplified = sp.simplify(twiddle_factor_half)
print("\nTwiddle factor for k+N/2:")
display(sp.Eq(twiddle_factor_half, twiddle_factor_half_simplified))

# Final relation for X[k+N/2]
X_k_half_recurrence = sp.Eq(X[k_plus_half], X_even_k + twiddle_factor_half_simplified * X_odd_k)
X_k_half_recurrence_simplified = sp.Eq(X[k_plus_half], X_even_k - twiddle_factor * X_odd_k)
print("\nRecurrence relation for X[k+N/2]:")
display(X_k_half_recurrence)
display(X_k_half_recurrence_simplified)

# Complexity analysis
print("\nComplexity Analysis:")
print("Traditional DFT: O(N²) operations")
print("FFT via divide and conquer: O(N log N) operations")


Original DFT formula:


Eq(X[k], Sum(exp(-2.0*I*pi*k*n/N)*x[n], (n, 0, N - 1)))


Split into even and odd indices:


Eq(X[k], Sum(exp(-4.0*I*pi*k*m/N)*x[2*m], (m, 0, N/2 - 1)) + Sum(exp(-2.0*I*pi*k*(2*m + 1)/N)*x[2*m + 1], (m, 0, N/2 - 1)))


Simplified exponential terms:


...

Eq(exp(-2.0*I*pi*(N/2 + k)/N), exp(-1.0*I*pi*(N + 2*k)/N))


Recurrence relation for X[k+N/2]:


Eq(X[N/2 + k], X_even_k + X_odd_k*exp(-1.0*I*pi*(N + 2*k)/N))

Eq(X[N/2 + k], X_even_k - X_odd_k*exp(-2.0*I*pi*k/N))


Complexity Analysis:
Traditional DFT: O(N²) operations
FFT via divide and conquer: O(N log N) operations
